# Задание 5

## 1. ContextTimer(0.3 балла)
Напишите менеджер контекста, который позволит засекать время выполнения блока кода с помощью конструкции with и выводить это время на экран по выходу из блока. Пример использования:

<code>
with Timer ():
    do_some_long_stuff()
</code>

Рекомендации: используйте стандартную библиотеку для работы со временем.

Протестируйте себя.

In [1]:
import time


class Timer:
    def __enter__(self):
        self.start_time = time.process_time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop_time = time.process_time()
        print('This block has been working for '
              f'{self.stop_time - self.start_time} s')
        return exc_type is None


Тестируемся на сортировке пузырьком

In [2]:
def bubble_sort(a):
    for i in range(len(a) - 2, -1, -1):
        for j in range(i + 1):
            if a[j] > a[j + 1]:
                a[j], a[j + 1] = a[j + 1], a[j]

In [3]:
a = list(reversed(range(10)))
with Timer():
    bubble_sort(a)

This block has been working for 5.9480000000000643e-05 s


In [4]:
a = list(reversed(range(100)))
with Timer():
    bubble_sort(a)

This block has been working for 0.0008144310000000043 s


Один и тот же экземпляр <code>Timer</code> можно использовать много раз (если он используется ек более чем одним менеджером контекста одновременно)

In [5]:
a = list(reversed(range(1000)))
t = Timer()
with t:
    bubble_sort(a)

This block has been working for 0.10925626300000002 s


In [6]:
a = list(reversed(range(10_000)))
with t:
    bubble_sort(a)

This block has been working for 8.982510861000002 s


Проверка на исключения

In [7]:
with Timer():
    raise Exception()

This block has been working for 1.1770000000410619e-05 s


Exception: 

## 2.Transaction (0.4 балла)
Вам необходимо написать менеджер контекстов, который позволит безопасно работать с транзакциями. Напишите класс Storage, в котором будут храниться какие-то данные в виде словаря. Эти данные должны быть закрытыми и их можно только читать через операцию []. У этого класса должен быть метод edit, который возвращает менеджер контекста, позволяющий редактировать исходный объект (опять же через []). При этом результаты редактирования записываются в исходный объект только если весь блок выполнился успешно. Пример использования:

<code>
s = Storage()
with s.edit() as se :
    se["a"] = 1
    may_be_an_exception_here()
</code>

Протестируйте себя.

In [8]:
class _ContextManager:
    def __init__(self, main_storage):
        self._main_storage = main_storage
        self._new_storage = {}

    def __getitem__(self, item):
        try:
            return self._new_storage[item]
        except KeyError:
            return self._main_storage[item]

    def __setitem__(self, key, value):
        self._new_storage[key] = value

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            return False

        for key, value in self._new_storage.items():
            self._main_storage[key] = value
        return True


class Storage:
    def __init__(self, *args, **kwargs):
        self._storage = dict(*args, **kwargs)

    def __len__(self):
        return len(self._storage)

    def __getitem__(self, item):
        return self._storage[item]

    def edit(self):
        return _ContextManager(self._storage)

    def __repr__(self):
        return f'Storage({repr(self._storage)})'

    def __str__(self):
        return f'Storage({str(self._storage)})'


In [9]:
s = Storage(one=1, two=2)
s

Storage({'one': 1, 'two': 2})

In [10]:
s['two']

2

In [12]:
s['three'] = 3

TypeError: 'Storage' object does not support item assignment

In [13]:
with s.edit() as se:
    se['three'] = 3
    print(se['three'])
    se['two'] = 'два'
    print(se['two'])
    print(s['two'])
print(s['two'])
print(s)

3
два
2
два
Storage({'one': 1, 'two': 'два', 'three': 3})


In [14]:
try:
    with s.edit() as se:
        se['four'] = 4
        print(se['four'])
        raise KeyboardInterrupt()
except KeyboardInterrupt:
    pass
print(s)

4
Storage({'one': 1, 'two': 'два', 'three': 3})


## Phone numbers (0.3)
Напишите регулярное выражение для распознавания телефонных номеров и протестируйте себя. Номера, которые должны распознаваться:
* 3-22-46
* +71239513749
* 71239513749
* 1239513749
* +7(123)-951-37-49
* +7(123)9513749
* +7-123-9513749
* +7-123-951-37-49

* 7(123)-951-37-49
* 7(123)9513749
* 7-123-9513749
* 7-123-951-37-49

* (123)-951-37-49
* (123)9513749
* 123-9513749
* 123-951-37-49

In [ ]:
<your code here>